In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 3
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


D:\Hu_code\ResNet18_ESC10\ESC10_kFold_135s\..\TrainDataPreprocessing.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 27.26 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_1S
            epoch = 90
            lr = 0.0032
        elif(TL==1):
            trainData = data_3S
            epoch = 90
            lr = 0.0032
        else:
            trainData = data_5S
            epoch = 90
            lr = 0.0032
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 1.5731767416000366 Train_acc: 30.062500%
Train Epoch: 2/90 Train_Loss: 1.4395341873168945 Train_acc: 49.437500%
Train Epoch: 3/90 Train_Loss: 1.3354769945144653 Train_acc: 59.250000%
Train Epoch: 4/90 Train_Loss: 0.7847515344619751 Train_acc: 68.125000%
Train Epoch: 5/90 Train_Loss: 0.6152200698852539 Train_acc: 71.562500%
Train Epoch: 6/90 Train_Loss: 0.5326775312423706 Train_acc: 76.187500%
Train Epoch: 7/90 Train_Loss: 0.5236991047859192 Train_acc: 78.687500%
Train Epoch: 8/90 Train_Loss: 0.7032454609870911 Train_acc: 79.937500%
Train Epoch: 9/90 Train_Loss: 0.6110475659370422 Train_acc: 82.062500%
Train Epoch: 10/90 Train_Loss: 0.5465582609176636 Train_acc: 80.875000%
Train Epoch: 11/90 Train_Loss: 0.

Train Epoch: 16/90 Train_Loss: 0.026227999478578568 Train_acc: 97.291672%
Train Epoch: 17/90 Train_Loss: 0.07630375772714615 Train_acc: 98.229172%
Train Epoch: 18/90 Train_Loss: 0.011679280549287796 Train_acc: 98.020836%
Train Epoch: 19/90 Train_Loss: 0.01972237415611744 Train_acc: 98.229172%
Train Epoch: 20/90 Train_Loss: 0.03692885860800743 Train_acc: 98.437508%
Train Epoch: 21/90 Train_Loss: 0.043689318001270294 Train_acc: 98.645836%
Train Epoch: 22/90 Train_Loss: 0.08377579599618912 Train_acc: 98.750008%
Train Epoch: 23/90 Train_Loss: 0.06134019047021866 Train_acc: 98.958336%
Train Epoch: 24/90 Train_Loss: 0.008456711657345295 Train_acc: 98.958336%
Train Epoch: 25/90 Train_Loss: 0.028790529817342758 Train_acc: 98.958336%
Train Epoch: 26/90 Train_Loss: 0.01023098360747099 Train_acc: 98.958336%
Train Epoch: 27/90 Train_Loss: 0.024384573101997375 Train_acc: 98.645836%
Train Epoch: 28/90 Train_Loss: 0.00995017308741808 Train_acc: 98.854172%
Train Epoch: 29/90 Train_Loss: 0.013059236109

Train Epoch: 32/90 Train_Loss: 0.0014196662232279778 Train_acc: 100.000000%
Train Epoch: 33/90 Train_Loss: 0.0020735214930027723 Train_acc: 100.000000%
Train Epoch: 34/90 Train_Loss: 0.010491127148270607 Train_acc: 99.687500%
Train Epoch: 35/90 Train_Loss: 0.0006946680368855596 Train_acc: 100.000000%
Train Epoch: 36/90 Train_Loss: 0.00033098566927947104 Train_acc: 100.000000%
Train Epoch: 37/90 Train_Loss: 0.00035544190905056894 Train_acc: 100.000000%
Train Epoch: 38/90 Train_Loss: 0.0012266181875020266 Train_acc: 100.000000%
Train Epoch: 39/90 Train_Loss: 0.00020497004152275622 Train_acc: 100.000000%
Train Epoch: 40/90 Train_Loss: 0.0008269809768535197 Train_acc: 100.000000%
Train Epoch: 41/90 Train_Loss: 0.0007211749325506389 Train_acc: 100.000000%
Train Epoch: 42/90 Train_Loss: 0.0005343370139598846 Train_acc: 100.000000%
Train Epoch: 43/90 Train_Loss: 7.848511449992657e-05 Train_acc: 100.000000%
Train Epoch: 44/90 Train_Loss: 0.0002120794088114053 Train_acc: 100.000000%
Train Epoch

Train Epoch: 47/90 Train_Loss: 0.09639280289411545 Train_acc: 94.062500%
Train Epoch: 48/90 Train_Loss: 0.1145152598619461 Train_acc: 94.375000%
Train Epoch: 49/90 Train_Loss: 0.1891692727804184 Train_acc: 94.000000%
Train Epoch: 50/90 Train_Loss: 0.07597692310810089 Train_acc: 94.375000%
Train Epoch: 51/90 Train_Loss: 0.18796397745609283 Train_acc: 94.250000%
Train Epoch: 52/90 Train_Loss: 0.08375276625156403 Train_acc: 94.250000%
Train Epoch: 53/90 Train_Loss: 0.13227394223213196 Train_acc: 94.250000%
Train Epoch: 54/90 Train_Loss: 0.19155016541481018 Train_acc: 94.062500%
Train Epoch: 55/90 Train_Loss: 0.10829812288284302 Train_acc: 94.375000%
Train Epoch: 56/90 Train_Loss: 0.09089884907007217 Train_acc: 93.812500%
Train Epoch: 57/90 Train_Loss: 0.18219730257987976 Train_acc: 94.250000%
Train Epoch: 58/90 Train_Loss: 0.0998498871922493 Train_acc: 94.062500%
Train Epoch: 59/90 Train_Loss: 0.04585738107562065 Train_acc: 94.062500%
Train Epoch: 60/90 Train_Loss: 0.1382782757282257 Trai

Train Epoch: 64/90 Train_Loss: 0.0002212728577433154 Train_acc: 98.854172%
Train Epoch: 65/90 Train_Loss: 0.03395303338766098 Train_acc: 98.854172%
Train Epoch: 66/90 Train_Loss: 0.04038364067673683 Train_acc: 98.854172%
Train Epoch: 67/90 Train_Loss: 0.00027365426649339497 Train_acc: 98.854172%
Train Epoch: 68/90 Train_Loss: 0.022556619718670845 Train_acc: 98.854172%
Train Epoch: 69/90 Train_Loss: 0.0008548512705601752 Train_acc: 98.854172%
Train Epoch: 70/90 Train_Loss: 0.036030083894729614 Train_acc: 98.854172%
Train Epoch: 71/90 Train_Loss: 0.023931222036480904 Train_acc: 98.854172%
Train Epoch: 72/90 Train_Loss: 0.02396518550813198 Train_acc: 98.854172%
Train Epoch: 73/90 Train_Loss: 0.023438304662704468 Train_acc: 98.854172%
Train Epoch: 74/90 Train_Loss: 9.869954374153167e-05 Train_acc: 98.958336%
Train Epoch: 75/90 Train_Loss: 0.019942089915275574 Train_acc: 98.854172%
Train Epoch: 76/90 Train_Loss: 0.0431092344224453 Train_acc: 98.958336%
Train Epoch: 77/90 Train_Loss: 0.01076

...

Train Epoch: 32/90 Train_Loss: 0.14326661825180054 Train_acc: 94.562500%
Train Epoch: 33/90 Train_Loss: 0.12678752839565277 Train_acc: 93.687500%
Train Epoch: 34/90 Train_Loss: 0.15551847219467163 Train_acc: 94.500000%
Train Epoch: 35/90 Train_Loss: 0.17274227738380432 Train_acc: 94.187500%
Train Epoch: 36/90 Train_Loss: 0.19903528690338135 Train_acc: 94.625000%
Train Epoch: 37/90 Train_Loss: 0.09004992246627808 Train_acc: 94.437500%
Train Epoch: 38/90 Train_Loss: 0.0651286318898201 Train_acc: 94.000000%
Train Epoch: 39/90 Train_Loss: 0.09148775041103363 Train_acc: 94.687500%
Train Epoch: 40/90 Train_Loss: 0.21405190229415894 Train_acc: 94.375000%
Train Epoch: 41/90 Train_Loss: 0.0844351127743721 Train_acc: 94.312500%
Train Epoch: 42/90 Train_Loss: 0.08224684745073318 Train_acc: 94.750000%
Train Epoch: 43/90 Train_Loss: 0.15929675102233887 Train_acc: 94.312500%
Train Epoch: 44/90 Train_Loss: 0.22501663863658905 Train_acc: 94.375000%
Train Epoch: 45/90 Train_Loss: 0.083619624376297 Trai

Train Epoch: 49/90 Train_Loss: 0.01651643216609955 Train_acc: 98.645836%
Train Epoch: 50/90 Train_Loss: 0.04955383762717247 Train_acc: 98.645836%
Train Epoch: 51/90 Train_Loss: 0.03629433363676071 Train_acc: 98.645836%
Train Epoch: 52/90 Train_Loss: 0.0071992091834545135 Train_acc: 98.645836%
Train Epoch: 53/90 Train_Loss: 0.049551017582416534 Train_acc: 98.645836%
Train Epoch: 54/90 Train_Loss: 0.02250591292977333 Train_acc: 98.541672%
Train Epoch: 55/90 Train_Loss: 0.08785132318735123 Train_acc: 98.750008%
Train Epoch: 56/90 Train_Loss: 0.05756167322397232 Train_acc: 98.750008%
Train Epoch: 57/90 Train_Loss: 9.569420217303559e-05 Train_acc: 98.645836%
Train Epoch: 58/90 Train_Loss: 0.03278081491589546 Train_acc: 98.750008%
Train Epoch: 59/90 Train_Loss: 0.020753338932991028 Train_acc: 98.645836%
Train Epoch: 60/90 Train_Loss: 0.021181734278798103 Train_acc: 98.645836%
Train Epoch: 61/90 Train_Loss: 0.01167264673858881 Train_acc: 98.750008%
Train Epoch: 62/90 Train_Loss: 0.02039342559

Train Epoch: 64/90 Train_Loss: 0.00011606969928834587 Train_acc: 100.000000%
Train Epoch: 65/90 Train_Loss: 0.00026722034090198576 Train_acc: 100.000000%
Train Epoch: 66/90 Train_Loss: 0.00018097017891705036 Train_acc: 100.000000%
Train Epoch: 67/90 Train_Loss: 7.61082919780165e-05 Train_acc: 100.000000%
Train Epoch: 68/90 Train_Loss: 0.0003135456645395607 Train_acc: 100.000000%
Train Epoch: 69/90 Train_Loss: 5.6046119425445795e-05 Train_acc: 100.000000%
Train Epoch: 70/90 Train_Loss: 0.0006382913561537862 Train_acc: 100.000000%
Train Epoch: 71/90 Train_Loss: 0.00012229452840983868 Train_acc: 100.000000%
Train Epoch: 72/90 Train_Loss: 0.001009819912724197 Train_acc: 100.000000%
Train Epoch: 73/90 Train_Loss: 0.0001489550486439839 Train_acc: 100.000000%
Train Epoch: 74/90 Train_Loss: 0.00019200598762836307 Train_acc: 100.000000%
Train Epoch: 75/90 Train_Loss: 0.00011840714432764798 Train_acc: 100.000000%
Train Epoch: 76/90 Train_Loss: 0.0005250161048024893 Train_acc: 100.000000%
Train E

Train Epoch: 81/90 Train_Loss: 0.061401642858982086 Train_acc: 94.750000%
Train Epoch: 82/90 Train_Loss: 0.17884935438632965 Train_acc: 94.750000%
Train Epoch: 83/90 Train_Loss: 0.022954579442739487 Train_acc: 94.937500%
Train Epoch: 84/90 Train_Loss: 0.11397358030080795 Train_acc: 94.812500%
Train Epoch: 85/90 Train_Loss: 0.19723080098628998 Train_acc: 94.687500%
Train Epoch: 86/90 Train_Loss: 0.07459557056427002 Train_acc: 94.437500%
Train Epoch: 87/90 Train_Loss: 0.1537991613149643 Train_acc: 94.812500%
Train Epoch: 88/90 Train_Loss: 0.11941996216773987 Train_acc: 94.812500%
Train Epoch: 89/90 Train_Loss: 0.12278673052787781 Train_acc: 94.875000%
Train Epoch: 90/90 Train_Loss: 0.1644548624753952 Train_acc: 94.812500%
Test accuracy of the model: 71.000000%
Test accuracy of the model: 62.916667%
Test accuracy of the model: 52.500000%
Training Time: 56.48 seconds
----------------------------------------------------------------------------------------------------------------
fold:  5 TL

Train Epoch: 3/90 Train_Loss: 0.03853555768728256 Train_acc: 96.875000%
Train Epoch: 4/90 Train_Loss: 0.05214199796319008 Train_acc: 98.125000%
Train Epoch: 5/90 Train_Loss: 0.12914302945137024 Train_acc: 98.125000%
Train Epoch: 6/90 Train_Loss: 0.11164248734712601 Train_acc: 98.750000%
Train Epoch: 7/90 Train_Loss: 0.020072434097528458 Train_acc: 100.000000%
Train Epoch: 8/90 Train_Loss: 0.011020274832844734 Train_acc: 99.687500%
Train Epoch: 9/90 Train_Loss: 0.005636046640574932 Train_acc: 98.437500%
Train Epoch: 10/90 Train_Loss: 0.17831820249557495 Train_acc: 97.500000%
Train Epoch: 11/90 Train_Loss: 0.18444301187992096 Train_acc: 97.812500%
Train Epoch: 12/90 Train_Loss: 0.007040370721369982 Train_acc: 98.125000%
Train Epoch: 13/90 Train_Loss: 0.00232015922665596 Train_acc: 98.750000%
Train Epoch: 14/90 Train_Loss: 0.005197381600737572 Train_acc: 100.000000%
Train Epoch: 15/90 Train_Loss: 0.0026249552611261606 Train_acc: 100.000000%
Train Epoch: 16/90 Train_Loss: 0.004976261407136

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()